# Modules

In [ ]:
import numpy as np
import scipy as sp
from scipy.optimize import curve_fit
import pickle
import MDAnalysis as mda
import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
import multiprocessing

In [ ]:
import memdiff

# Mean square displacement

In [ ]:
def plot_msd(name):
    
    # Load data
    with open('../results/martini-bigsimulations/diffusion/diffusion_clusters_'+name+'.p','br') as infile:
        cluster_sizes, cluster_dc, cluster_err, time, cluster_msd, cluster_points = pickle.load(infile)

    fig, ax = plt.subplots(1, 1, figsize=plt.figaspect(.75)*1.0, dpi=100) # , sharex='col', sharey='row'

    rainbow = plt.get_cmap('rainbow')
    for n,cs in enumerate(cluster_sizes):
        msd_n = cluster_msd[n,:]*0.1 # conversion to nm
        if cs==1 or cs==np.max(cluster_sizes):
            ax.plot(time[:len(msd_n)],msd_n,color=rainbow(np.sqrt(float(cs)/np.max(cluster_sizes))),label='cluster size $s_c = $'+str(cs))
        else:
            ax.plot(time[:len(msd_n)],msd_n,color=rainbow(np.sqrt(float(cs)/np.max(cluster_sizes))))

    ax.set_xlim([0,20])
    ax.set_ylim([0,20])

    ax.set_xlabel(r'$t\;[\mathrm{ns}]$',fontsize=20)
    ax.set_ylabel(r'MSD$(t)\;[\mathrm{nm^2}]$',fontsize=20)

    ax.legend(loc='upper left',fontsize=14)
    
    ax.tick_params(labelsize=14)

    fig.tight_layout()

#    fig.savefig('../plots/msd_'+name+'.pdf', fmt='pdf', dpi=300)
    
    return

In [ ]:
plot_msd('POPC-100CNT-12-8-f11-SNda-A')

# Combined Fit of Diffusion Coefficients

In [ ]:
kB = 1.38064852e-23
eta_f_martini = 10.2e-4
r_CNT = 1.05 # radius of a CNT in nm


def d_SD_visc( x, eta_m, eta_f=eta_f_martini, T=300 ): 
    """ Saffman-Delbrueck law from viscosities
    Arguments:
        x: radius of the diffusing object in nm
        eta_m: membrane surface viscosity in Pa s m
        eta_f: water viscosity in Pa s
        T: temperature in K
    Returns:
        dc: diffusion coefficient predicted by the SD law in 10^-11 m^2/s resp. 10^-7 cm^2/s
    """
    
    a   = kB*T/(4*np.pi*eta_m)
    b   = 1.0e9*0.5*eta_m/eta_f 
    eps = x/b
    
    dc  = a * ( np.log( 2./eps ) - np.euler_gamma )
    
    return 1.0e11*dc


def d_PS_visc( x, eta_m, eta_f=eta_f_martini, T=300 ):
    """ Saffman-Delbrueck law from viscosities
    Arguments:
        x: radius of the diffusing object in nm
        eta_m: membrane surface viscosity in Pa s m
        eta_f: water viscosity in Pa s
        T: temperature in K
    Returns:
        dc: diffusion coefficient predicted by the HPW law in 10^-11 m^2/s resp. 10^-7 cm^2/s
    """
    
    a = kB*T/(4*np.pi*eta_m)
    b = 1.0e9*0.5*eta_m/eta_f # SD length in nm
    
    c1  = 0.73761 
    b1  = 2.74819 
    c2  = 0.52119 
    b2  = 0.51465 
    eps = x/b
    
    dc  = a * ( np.log(2./eps) - np.euler_gamma + 4*eps/np.pi - (eps**2/2.)*np.log(2./eps) )
    dc /= 1 - (eps**3/np.pi)*np.log(2./eps) + c1*eps**b1/( 1 + c2*eps**b2 )
    
    return 1.0e11*dc

def d_SE_visc( x, eta_m, eta_f=eta_f_martini, T=300 ):
    """ Stokes Einstein law from viscosities
    Arguments:
        x: radius of the diffusing object in nm
        eta_m: membrane surface viscosity in Pa s m
        eta_f: water viscosity in Pa s
        T: temperature in K
    Returns:
        dc: diffusion coefficient predicted by the Stokes-Einstein-like law in 10^-11 m^2/s resp. 10^-7 cm^2/s
    """
    
    a = kB*T/(4*np.pi*eta_m)
    b = 1.0e9*0.25*eta_m/eta_f # SD length in nm
    
    c1  = 0.73761 
    b1  = 2.74819 
    c2  = 0.52119 
    b2  = 0.51465 
    eps = x/b
    
    dc  = a / ( eps/np.pi  )
    
    return 1.0e11*dc


def d_corr_visc( d_sim, eta_m ):
    """ Saffman-Delbrueck law from viscosities
    Arguments:
        d_sim: diffusion coefficient from the simulation
        eta_m: membrane surface viscosity in Pa s m
        eta_f: water viscosity in Pa s
        T: temperature in K
    Returns:
        dc: diffusion coefficient predicted by the HPW law
    """

    d_corr = memdiff.d0_approximation(dpbc=d_sim*1.0e-7,eta_f=eta_f_martini,eta_m=eta_m,h=2.5,l=70,T=300)
    
    return 1.0e7*d_corr

In [ ]:
name = 'POPC-100CNT-12-8-f11-SNda-all'

with open('../results/martini-bigsimulations/diffusion/diffusion_blocks_'+name+'.p','br') as infile:
    block_cluster_size_avg, block_cluster_size_std, block_cluster_asym_avg, block_cluster_asym_std, block_cluster_radius_avg, block_cluster_radius_std, block_dc = pickle.load(infile)

r = [] # effective radius
d = [] # diffusion coefficient
e = [] # error of the diffusion coefficient

for i in np.arange(100):

    valid = block_cluster_size_avg==i+1  
    num_samples = np.sum(valid)
    
    if num_samples > 40:
        
        dc = block_dc[valid]

        new_r = np.sqrt(i+1) 
        new_d = np.average(dc)

        # Error estimate via corrected standard error of the mean
        new_e = np.std(dc)*np.sqrt((i+1)/(float(num_samples)))

        if np.isfinite(new_e):

            r.append( new_r )
            d.append( new_d )
            e.append( new_e )

r = np.array(r) * r_CNT # multiply with radius of one CNT
d = np.array(d)
e = np.array(e)  

In [ ]:
# --------------------------- #
#   Fit to uncoreected data   #
# --------------------------- #

eta_m_fit_raw, pcov = sp.optimize.curve_fit( d_PS_visc,r,d, p0=[4e-11], sigma=e, absolute_sigma=True )
l_sd = 0.5e9*eta_m_fit_raw[0]/eta_f_martini

# Goodness of the fit
chi2   = np.sum( ((d-d_PS_visc(r,eta_m_fit_raw))/e)**2 )
nu     = len(r)-2
chi2nu = chi2/nu

# Result
# Result
print('Fit to the raw data:')
print('eta_m = %3.3e Pa s m'% eta_m_fit_raw[0])
print('L_SD = %.1f nm' % l_sd )
print('chi^2 = %.1f +/- %.1f' % (chi2nu,np.sqrt(2./nu)))
print('')


# -------------------------------- #
#   Fit including the correction   #
# -------------------------------- #

def fit_correction(x, eta_m_fit):
    return d_corr_visc( d, eta_m_fit) - d_PS_visc( x, eta_m_fit )
eta_m_fit_corr, pcov = sp.optimize.curve_fit( fit_correction,r,d*0, p0=[4e-11], sigma=e, absolute_sigma=True )
l_sd = 0.5e9*eta_m_fit_corr[0]/eta_f_martini

# Goodness of the fit
chi2   = np.sum( (fit_correction(r,eta_m_fit_corr)/e)**2 )
nu     = len(r)-2
chi2nu = chi2/nu

# Result
print('Fit to the corrected data:')
print('eta_m = %3.3e Pa s m'% eta_m_fit_corr)
print('L_SD = %.1f nm' % l_sd )
print('chi^2 = %.1f +/- %.1f' % (chi2nu,np.sqrt(2./nu)))

In [ ]:
fig, ax = plt.subplots(2,1, figsize=[3.5,6.2], dpi=100) # , sharey='row', sharex='col'


# ---------------------------- #
#   Plots of data and theory   #
# ---------------------------- #

# D from simulations
ax[0].plot( r, d, marker='s', ls='none', color='C0', mec='none', label='simulated' )
ax[0].errorbar( r, d, e, ls='none', color='C0', mec='none', alpha=0.5 )

# Corrected D
ax[0].plot( r, d_corr_visc( d, 4.3e-11 ), marker='o', ls='none', color='C1', mec='none', label='corrected' )
ax[0].errorbar( r, d_corr_visc( d, 4.3e-11 ), e, ls='none', color='C1', mec='none', alpha=0.5 )

# Theory with fitted eta_m
xax = np.arange(0.5,13,0.1)
#ax.plot( xax, d_PS_visc(xax,eta_m_fit_raw ), ':', lw=3, color='C5', label=r'fit to sim.' )
#ax.plot( xax, d_PS_visc(xax,eta_m_fit_corr), '--', lw=3, color='C3', label=r'fit to corr.' )
ax[0].plot( xax, d_PS_visc(xax,eta_m_fit_corr), '-', lw=3, color='C3', label=r'HPW-PS theory' )
ax[0].plot( xax, d_SD_visc(xax,eta_m_fit_corr), ':',  lw=3, color='C2', label=r'SD theory' )

ax[0].legend(loc='upper right') #,fontsize=14)
ax[0].set_xlim([0.5,12])
ax[0].set_ylim([0,2.75])
ax[0].set_xlabel(r'effective cluster radius') #,fontsize=20)
ax[0].set_ylabel(r'$D\;[10^{-7}\,\mathrm{cm^2\,s^{-1}}]$') #,fontsize=20)


# ----------------------------- #
#  Plots of example MSD curves  #
# ----------------------------- #

# Load data
with open('../results/martini-bigsimulations/diffusion/diffusion_clusters_POPC-100CNT-12-8-f11-SNda.p','br') as infile:
    cluster_sizes, cluster_dc, cluster_err, time, cluster_msd, cluster_points = pickle.load(infile)

rainbow = plt.get_cmap('viridis')
print('Cluster sizes:', cluster_sizes)
for n,cs in enumerate(cluster_sizes):
    msd_n = cluster_msd[n,:]*0.1 # conversion to nm
    if cs==1 or cs==np.max(cluster_sizes):
        ax[1].plot(time[:len(msd_n)],msd_n,color=rainbow(np.sqrt(float(cs)/np.max(cluster_sizes))),label='cluster size $s_c = $'+str(cs))
    else:
        ax[1].plot(time[:len(msd_n)],msd_n,color=rainbow(np.sqrt(float(cs)/np.max(cluster_sizes))))

ax[1].set_xlim([0,17.5])
ax[1].set_ylim([0,17.5])

ax[1].set_xlabel(r'$t\;[\mathrm{ns}]$') #,fontsize=20)
ax[1].set_ylabel(r'MSD$(t)\;[\mathrm{nm^2}]$') #,fontsize=20)

ax[1].legend(loc='upper left') #,fontsize=14)
    
        
# ---------------------------- #
#   Labels, Style and Layout   #
# ---------------------------- #

panellabel = ["A","B"]
for i,axi in enumerate(ax):
    #axi.tick_params(labelsize=16)
    axi.text(0.96, 0.12, panellabel[i], transform=axi.transAxes, fontsize=16, va='top', ha='right') # , fontweight='bold'

fig.tight_layout()

fig.savefig("figure3.pdf", format='pdf', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=plt.figaspect(.75)*1.0, dpi=100) # , sharey='row', sharex='col'

# ---------------------------- #
#   Plots of data and theory   #
# ---------------------------- #

# D from simulations
ax.plot( r, d, marker='s', ls='none', color='C0', mec='none', label='simulated' )
ax.errorbar( r, d, e, ls='none', color='C0', mec='none', alpha=0.5 )

# Corrected D
ax.plot( r, d_corr_visc( d, 4.3e-11 ), marker='o', ls='none', color='C1', mec='none', label='corrected' )
ax.errorbar( r, d_corr_visc( d, 4.3e-11 ), e, ls='none', color='C1', mec='none', alpha=0.5 )

# Theory with fitted eta_m
xax = np.arange(0.5,13,0.1)
#ax.plot( xax, d_PS_visc(xax,eta_m_fit_raw ), ':', lw=3, color='C5', label=r'fit to sim.' )
#ax.plot( xax, d_PS_visc(xax,eta_m_fit_corr), '--', lw=3, color='C3', label=r'fit to corr.' )
ax.plot( xax, d_PS_visc(xax,eta_m_fit_corr), '-', lw=3, color='C3', label=r'HPW-PS theory' )
ax.plot( xax, d_SD_visc(xax,eta_m_fit_corr), ':',  lw=3, color='C2', label=r'SD theory' )
       
# ---------------------------- #
#   Labels, Style and Layout   #
# ---------------------------- #

ax.legend(loc='upper right',fontsize=14)
ax.tick_params(labelsize=14)
ax.set_xlim([0,12.5])
ax.set_ylim([0,2.75])
ax.set_xlabel(r'effective cluster radius',fontsize=20)
ax.set_ylabel(r'$D\;[10^{-7}\,\mathrm{cm^2\,s^{-1}}]$',fontsize=20)
fig.tight_layout()

fig.savefig("figure3a.pdf", format='pdf')

Diffusing clusters behave like one big object and follow the Petrov-Schwille relation. The size-corrected diffusion coefficients match quantitatively to the theoretical prediction. Even though we do not exactly know the membrane viscosity and allow it as a free fit parameter, we can see that they match only badly. However fitting the theory including the finite-size correction gives a perfect agreement. This insight will help to refine mesoscale models [citation Duncan/Chavent et al.] and the analysis of diffusion in crowded membranes [cite Javanainen et al.]. 

The range of validity of the SD law is much smaller in Martini. The viscosities of the membrane and of the water do not scale in the same way with respect to real systems (and to atomistic simulations). Therefore $L_{SD}$, the characteristic length scale, is only tens of nanometers in Martini instead of hundreds. It is a priori clear that absolute dynamics in coarse-grained models is different from the situation in atomistic systems, however care should also be taken when comparing relative dynamics. A shift in the effective length scale of the system can lead to qualitatively different behavior.

In [ ]:
l_sd = 0.5e9*eta_m_fit_corr/eta_f_martini

fig, ax = plt.subplots(1, 1, figsize=plt.figaspect(.75)*1.0, dpi=100) # , sharey='row', sharex='col'

a = np.arange(0.8,500)
ax.plot( a, d_PS_visc(a,4.5e-11),  '-', lw=3, color='black', label='PS')
ax.plot( a, d_SD_visc(a,4.5e-11),  ':', lw=3, color='black', label='SD')
ax.plot( a, d_SE_visc(a,4.5e-11), '--', lw=3, color='black', label='SE')
ax.vlines( l_sd, 0, 3, lw=3, color='grey')

ax.legend(loc='upper right',fontsize=14)
ax.annotate(r'$L_\mathrm{SD}$',[l_sd+3.,2.],size=14,color='grey')

ax.tick_params(labelsize=14)
ax.set_xlim(0.8,500)
ax.set_ylim(0.0,2.75)
ax.set_xscale('log')
#ax.set_yscale('log')
ax.set_xlabel(r'effective radius',fontsize=20)
ax.set_ylabel(r'$D\;[10^{-7}\,\mathrm{cm^2\,s^{-1}}]$',fontsize=20)
fig.tight_layout()